**CAT VS RABBIT**

IMPORTING THE LIBRARIES

In [ ]:
import os
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

import tensorflow_hub as hub

IMPORT THE DATASET

In [ ]:
data_directory = pathlib.Path('/kaggle/input/cat-vs-rabbit/val-cat-rabbit')
class_names = [item.name for item in data_directory.glob('*')][:2]
print(class_names)

In [ ]:
rabbit_dir="/kaggle/input/cat-vs-rabbit/val-cat-rabbit/rabbit"
cat_dir="/kaggle/input/cat-vs-rabbit/val-cat-rabbit/rabbit"
data_dir="/kaggle/input/cat-vs-rabbit/val-cat-rabbit"

NUMBER OF CAT AND RABBIT IMAGES

In [ ]:
num_rabbit_images = len(os.listdir(data_dir))
num_cat_images = len(os.listdir(cat_dir))
print(f'Number of rabbit images: {num_rabbit_images}')
print(f'Number of cat images: {num_cat_images}')

DATA GENERATOR

In [ ]:
data_gen = ImageDataGenerator(rescale = 1/255., validation_split = 0.2)

train_data = data_gen.flow_from_directory(data_dir, 
                                          target_size = (224, 224), 
                                          batch_size = 32,
                                          subset = 'training',
                                          class_mode = 'binary')
val_data = data_gen.flow_from_directory(data_dir, 
                                        target_size = (224, 224), 
                                        batch_size = 32,
                                        subset = 'validation',
                                        class_mode = 'binary')

In [ ]:
images, labels = train_data.next()
len(images), len(labels), images[0].shape

PLOTTING THE IMAGES

In [ ]:
def plot_random_images():
    """
    Plots 35 Random Images from the dataset
    """
    
    images, labels = train_data.next()
    labels = labels.astype('int32')
    i = 1

    plt.figure(figsize = (10, 10))
    
    for image, label in zip(images, labels):
        plt.subplot(4, 5, i)
        plt.imshow(image)
        plt.title(class_names[label])
        plt.axis('off')
        i += 1
        if i == 21:
            break

    plt.tight_layout()       
    plt.show()

In [ ]:
plot_random_images()

TRAINING THE DATA

In [ ]:
train_data, val_data

In [ ]:
model = tf.keras.Sequential([
            tf.keras.Input(shape = images[0].shape),
            tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2), strides=2),
            tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2), strides=2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation = 'relu'),
            tf.keras.layers.Dense(2, activation = 'softmax'),
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor = 'val_acc', patience = 3, verbose = 1, factor = 0.5, min_lr = 0.00001)

FITTING THE MODEL

In [ ]:
classifier = model.fit(train_data, batch_size = 32, 
                                 epochs = 12, 
                                 validation_data = val_data, 
                                 verbose = 1,
                                 callbacks = [reduce_lr])

In [ ]:
model.evaluate(val_data)

In [ ]:
def model_performance_graphs():
    
    fig, axes = plt.subplots(1, 2, figsize = (15, 8))

    axes[0].plot(classifier.epoch, classifier.history['accuracy'], label = 'acc')
    axes[0].plot(classifier.epoch, classifier.history['val_accuracy'], label = 'val_acc')
    axes[0].set_title('Accuracy vs Epochs', fontsize = 20)
    axes[0].set_xlabel('Epochs', fontsize = 15)
    axes[0].set_ylabel('Accuracy', fontsize = 15)
    axes[0].legend()

    axes[1].plot(classifier.epoch, classifier.history['loss'], label = 'loss')
    axes[1].plot(classifier.epoch, classifier.history['val_loss'], label="val_loss")
    axes[1].set_title("Loss Curve",fontsize=18)
    axes[1].set_xlabel("Epochs",fontsize=15)
    axes[1].set_ylabel("Loss",fontsize=15)
    axes[1].legend()

    plt.show()
    
model_performance_graphs()

    

In [23]:
model.save("CNN.h5")

In [24]:
loaded_model = tf.keras.models.load_model('/kaggle/working/CNN.h5')
loaded_model

In [25]:
def preprocess_images(path):
    img = image.load_img(path, target_size=(224, 224))

    img_array = image.img_to_array(img)

    img_array = np.expand_dims(img_array, axis=0)

    img_preprocessed = img_array / 255.0
    
    return img_preprocessed

PREDICTION WHETHER CAT OR RABBIT

In [26]:
img_path ="/kaggle/input/cat-vs-rabbit/test-images/cat/cat.234.jpg"
img = preprocess_images(img_path)
prediction = loaded_model.predict(img)

predicted_class_index = np.argmax(prediction)
class_labels = ['cat', 'rabbit']  
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class:", predicted_class_label)




1/1 [==============================] - 0s 66ms/step
Predicted class: cat
